In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
%matplotlib inline

In [3]:
from ikapati.visualization import visualize as vz
from ikapati.data import io

In [4]:
model_dir_path = "../models/7feacba0-2be0-4761-ba49-854067714f3b/"
model_dir = pathlib.Path(model_dir_path)
training_csv = model_dir.joinpath("training.csv")
data_dir = pathlib.Path("../data/processed")
train_dir = data_dir.joinpath("all")
sub_dirs = list(filter(lambda x: x.is_dir(), model_dir.iterdir()))

In [5]:
training_logs = pd.read_csv(training_csv)

In [6]:
def make_paths(paths):
    return list(map(lambda p: pathlib.Path("..", p), paths))

In [7]:
model_dir_paths = make_paths(training_logs["model_dir_path"].values)

In [8]:
def is_checkpoint(item):
    return item.stem.isnumeric()

def get_checkpoints(directory):
    return list(filter(is_checkpoint, list(directory.iterdir())))

def get_best_checkpoint(checkpoints):
    checkpoint_numbers = list(map(lambda x: int(x.stem), checkpoints))
    return np.argmax(checkpoint_numbers) if checkpoint_numbers else None

def get_best_checkpoint_in_directory(directory):
    checkpoints = get_checkpoints(directory)
    best_checkpoint = get_best_checkpoint(checkpoints)
    return checkpoints[best_checkpoint] if best_checkpoint else None

def get_best_checkpoint_files(sub_dirs):
    checkpoint_paths = []
    for sub_dir in sub_dirs:
        items = list(sub_dir.iterdir())
        best_checkpoint = get_best_checkpoint_in_directory(sub_dir)
        if best_checkpoint:
            checkpoint_paths.append(best_checkpoint)
    return checkpoint_paths

In [9]:
def evaluate_model(model_file_path):
    metadata_file_path = model_file_path.parent.joinpath("metadata.json")
    metadata = io.read_metadata(str(metadata_file_path))
    batch_size = metadata["arguments"]["batch_size"]
    test_data_path = train_dir.joinpath("test.tfrecord")
    test_dataset = io.read_dataset(str(test_data_path), batch_size, metadata["dataset"]["num_classes"])
    steps = metadata["dataset"]["file_counts"]["test"] // batch_size
    model = tf.keras.models.load_model(str(model_file_path))
    results = model.evaluate(test_dataset, steps=steps)
    print(results)
    return model, results

In [ ]:
results = []

for model_dir_path in model_dir_paths:
    print(str(model_dir_path))
    model_file_path = get_best_checkpoint_in_directory(model_dir_path)
    if not model_file_path:
        print(f"{str(model_dir_path)} : No model found, skipping.")
        continue
    model, result = evaluate_model(model_file_path)
    results.append([str(model_dir_path), result])
    model = None

../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__02_1926
../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__02_1926 : No model found, skipping.
../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__08_2457
 93/165 [===============>..............] - ETA: 20s - loss: 0.4582 - accuracy: 0.8873

In [18]:
model = None

In [33]:
model_dir_paths

[PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__02_1926'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__08_2457'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__19_3845'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__20_2543'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__21_5614'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-08__23_4632'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__01_1232'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__02_2409'),
 PosixPath('../models/7feacba0-2be0-4761-ba49-854067714f3b/2020-02-09__04_1552')]